### Implementation of Content-based Filtering

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import pickle

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

random.seed(42)


In [2]:
data_folder = 'Dataset/'
user_item_ratings_reduced = pd.read_csv(f'{data_folder}user_ratings_reduced.csv')
user_item_ratings_reduced = user_item_ratings_reduced.drop(columns=["Unnamed: 0"])
item_ratings_df = pd.read_csv(data_folder + 'user_ratings.csv')
item_info = pd.read_csv(data_folder + 'games.csv')
item_mecahnics = pd.read_csv(data_folder + 'mechanics.csv')
item_subcategories = pd.read_csv(data_folder + 'subcategories.csv')
item_themes = pd.read_csv(data_folder + 'themes.csv')

In [3]:
# user_item_ratings_reduced['rating'] = user_item_ratings_reduced['rating'].round(0).astype(int)

In [4]:
user_item_rating_count = user_item_ratings_reduced.groupby('user').count()

In [5]:
min_rating_count = 10
user_rating_count = user_item_rating_count[user_item_rating_count['rating'] >= min_rating_count]
user_list_filtered = list(user_rating_count.index)
user_item_ratings_filtered = user_item_ratings_reduced[user_item_ratings_reduced['user'].isin(user_list_filtered)]

In [6]:
user_item_ratings_filtered.head()

,item,rating,user
1,203416,8.0,-=Yod@=-
2,271896,8.0,-=Yod@=-
3,38054,6.0,-=Yod@=-
4,312267,5.0,-=Yod@=-
5,68448,9.0,-=Yod@=-


### Preprocessing

In [7]:
item_info.rename(columns={'BGGId':'item'},inplace=True)
# get comeplete user-catalog data
user_item_complete = pd.merge(user_item_ratings_filtered, item_info, on='item', how='inner')
# item_info.info() 
# item_info[['GoodPlayers', 'Family', 'ImagePath']].head()
non_numerical_features = ['user','Name','Description', 'item', 'GoodPlayers', 'Family', 'ImagePath']

In [8]:
# find a median rating for each user
median_rating = user_item_complete.groupby('user')['rating'].median().rename('median_rating').reset_index()
user_item_complete = pd.merge(user_item_complete, median_rating, on='user', how='inner')

In [9]:
user_item_complete.shape

(1228331, 51)

User-Item matrix has size (1_228_331, 51) after selecting users with >= `min_rating_count`

User-Item matrix has size (771_551, 51) after selecting items for users with rating higher than their mean rating

In [10]:
# creating user represenations
user_item_favourite = user_item_complete[user_item_complete['rating']>=user_item_complete['median_rating']]

user_item_favourite['ComAgeRec'].fillna(user_item_favourite['ComAgeRec'].mean(), inplace=True)
user_item_favourite['LanguageEase'].fillna(user_item_favourite['LanguageEase'].mean(), inplace=True)
user_item_favourite = user_item_favourite.drop(['Name', 'Description', 'Family', 'ImagePath'], axis=1)

/var/folders/hc/hnq3mts53txg3k7xbmj90z_80000gn/T/ipykernel_14743/2084522932.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_favourite['ComAgeRec'].fillna(user_item_favourite['ComAgeRec'].mean(), inplace=True)
/var/folders/hc/hnq3mts53txg3k7xbmj90z_80000gn/T/ipykernel_14743/2084522932.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_favourite['LanguageEase'].fillna(user_item_favourite['LanguageEase'].mean(), inplace=True)


In [11]:
user_representation = user_item_favourite.groupby('user').median(numeric_only=True).reset_index()
user_representation_wnames = user_representation['user']
user_representation.drop(columns=['user'], inplace=True)

In [12]:
user_representation.shape

(51149, 45)

### Analysis of user preference clusters
 > based on TSNE clusterings of users with similar preferences, several groups of users were investigated to check if there are differences in prederences between clusters. The analysis showed that all the groups have approximately similar preferences. Hence, no need to create different features per user group

In [13]:
def get_all_items_for_user(username, complete_df):
    return complete_df[complete_df['user']==username].drop(columns=['user'])

In [14]:
def add_avg_group_rating(selected_items, group, complete):
    selected_items.reset_index(drop=True, inplace=True)
    selected_items['avg_group_rating'] = 0
    group_users = group['user'].unique()
    for index,item in selected_items.iterrows():
        item_in_group = complete[(complete['user'].isin(group_users))&(complete['item']==item['item'])]
        average_group_rating = item_in_group['rating'].mean()
        item['avg_group_rating'] = average_group_rating
        selected_items.iloc[index,:] = item
        
    return selected_items


In [15]:
all_users = pd.merge(user_representation, user_representation_wnames,left_index=True, right_index=True, how='left')
all_items = add_avg_group_rating(user_item_complete.sample(frac=0.005, random_state=42), all_users, user_item_complete)

/var/folders/hc/hnq3mts53txg3k7xbmj90z_80000gn/T/ipykernel_14743/2979002713.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '7.4199098428453265' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  selected_items.iloc[index,:] = item


In [16]:
def save(obj):
    def get_variable_name(var):
        # Iterate over the global variables
        for name, value in globals().items():
            if value is var:
                return name
        return None
    name = get_variable_name(obj)
    with open(name+".pkl", "wb") as f: # "wb" because we want to write in binary mode
        pickle.dump(obj, f)
        
def open(path):
    return pickle.loads(path)

In [17]:
def test_stat_significance(feature_name, selected_items):
    """ test whether an item feature has statistically significant 
        correlation (Pearson) with a group rating
    """

    feature = selected_items[feature_name].dropna()
    ratings = selected_items['avg_group_rating'].reindex(feature.index).values
    feature = feature.values
    
    r, p = pearsonr(ratings, feature)
    
    significant = False
    if p < 0.05:
        significant = True

    # print(f'feature {feature_name} has p-value {round(p,2)}, significant: {significant}, correlation: {round(r,2)}')
    return r, significant


In [18]:
def find_group_features(group_items, complete):
    features = group_items.columns
    group_features = []
    for feature_name in features:
        if feature_name in non_numerical_features:
            continue
        
        correlation, significant = test_stat_significance(feature_name, group_items)
        
        if significant and abs(correlation) > 0.10:
            group_features.append(feature_name)
    return group_features

In [19]:
sign_user_preferences = set(find_group_features(all_items, user_item_complete))

/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [20]:
rating_features = ['AvgRating', 'BayesAvgRating', 'StdDev', 'avg_group_rating', 'median_rating']

### Item Features for Content-Based Filtering
- the features which have correlation with an average rating more than 0.15 in absolute values
- are not rating features
- are statistically significant


The features (19) were received:
```
 'Rank:boardgame',
 'Rank:strategygames',
 'Rank:childrensgames',
 'Rank:partygames',

 'NumWant',
 'NumWish',
 'NumUserRatings',
 'NumOwned',
 'NumWeightVotes',

 'BestPlayers',
 'GameWeight',
 'ComAgeRec',
 'Kickstarted',
 'MinPlayers',
 'LanguageEase',
 'MfgAgeRec'

 'Cat:Party',
 'Cat:Childrens',
 'Cat:Strategy',
 ```

Based on the analysis of the correlation of similar features from the list, the following were discarded:

 `'NumWish', 'NumWeightVotes'`

In [21]:
final_item_features = [f for f in (list(sign_user_preferences) + ['item','user']) if f not in rating_features + ['NumWish','NumWeightVotes']]

In [22]:
mechanics_features = ['mechanic_' + i if not i == 'BGGId' else i for i in item_mecahnics.columns]
subcategory_features = ['subcategory_' + i if not i == 'BGGId' else i for i in item_subcategories.columns]
theme_features = ['theme_' + i.split('_')[-1] if not i == 'BGGId' else i for i in item_themes.columns]

item_mecahnics.columns = mechanics_features
item_subcategories.columns = subcategory_features
item_themes.columns = theme_features

item_catalog = pd.merge(item_mecahnics, item_subcategories, on='BGGId', how='outer')
item_catalog = pd.merge(item_catalog, item_themes, on='BGGId', how='outer')

In [23]:
user_item_selected = user_item_complete[final_item_features]
user_item_all_features = pd.merge(user_item_selected, item_catalog, left_on='item', right_on='BGGId', how='left')

In [24]:
user_item_all_features.select_dtypes(include=['object']).columns

Index(['user'], dtype='object')

In [25]:
all_user_names = user_item_all_features.user.unique()

In [26]:
user_item_all_features[final_item_features][user_item_all_features['item']==5467]

,MfgAgeRec,ComMaxPlaytime,NumUserRatings,NumExpansions,GameWeight,Rank:strategygames,Rank:childrensgames,BestPlayers,Cat:Family,MinPlayers,...,ComAgeRec,Cat:Strategy,Kickstarted,LanguageEase,Rank:boardgame,Cat:Party,rating,ComMinPlaytime,item,user
227086,8,45,175,0,1.875,21926,21926,0,0,3,...,NaN,0,0,NaN,10921,0,5.0,45,5467,BoardGameDude
333010,8,45,175,0,1.875,21926,21926,0,0,3,...,NaN,0,0,NaN,10921,0,8.0,45,5467,Aquafan
442881,8,45,175,0,1.875,21926,21926,0,0,3,...,NaN,0,0,NaN,10921,0,7.0,45,5467,KlausScharpf
808612,8,45,175,0,1.875,21926,21926,0,0,3,...,NaN,0,0,NaN,10921,0,6.0,45,5467,Disneyphile
1046734,8,45,175,0,1.875,21926,21926,0,0,3,...,NaN,0,0,NaN,10921,0,6.0,45,5467,Snowy0505
1174991,8,45,175,0,1.875,21926,21926,0,0,3,...,NaN,0,0,NaN,10921,0,8.0,45,5467,caffeinatedgamer


# Group Recommender System

## Dividing users in to groups
Groups are made by looking at the most similar users
The features are aggregated by addition strategy

In [27]:
user_item_ratings_filtered.head()

,item,rating,user
1,203416,8.0,-=Yod@=-
2,271896,8.0,-=Yod@=-
3,38054,6.0,-=Yod@=-
4,312267,5.0,-=Yod@=-
5,68448,9.0,-=Yod@=-


## Dividing group using synthetic group generation function provided in lab2 RecSys

In [28]:
# from synthetic_groups_generation.groups_generators import GroupsGenerator
#
# user_matrix = user_item_ratings_filtered.pivot_table(columns='item', index='user', values='rating')
# user_matrix= user_matrix.iloc[:10000]
# user_id_set = set(user_matrix.index.values)
# user_id_indexes = user_matrix.index
# user_matrix = user_matrix.fillna(0)

In [29]:
# user_matrix.to_csv("user_matrix.csv")

In [30]:
# user_matrix = pd.read_csv("user_matrix.csv")

In [31]:
# numpy_array = user_matrix.to_numpy()
# sim_matrix = np.corrcoef(numpy_array[:][1:])

In [32]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(sim_matrix.flatten())
# plt.yscale("log")
# plt.xlabel("Pearson's Correlation Coeficient")

In [33]:
# grpGenerator = GroupsGenerator.getGroupsGenerator("SIMILAR")
# current_list = grpGenerator.generateGroups(user_id_indexes, sorted(user_id_set), sim_matrix, [4], 250)
#
# display(pd.DataFrame.from_records(current_list))
# groups = pd.DataFrame.from_records(current_list)

In [34]:
# groups_ratings = pd.DataFrame(columns=final_item_features)
#
# # For each group aggregate ratings
# for i in range(len(groups)):
#     # Get group members usernames
#     selected_users = groups.iloc[i]["group_members"]
#
#     # Create a dataframe for particular group
#     group_ratings = pd.DataFrame(columns=final_item_features)
#     # Append their ratings
#     for user in selected_users:
#         selected_users_rating = user_item_all_features[user_item_all_features["user"]==user][final_item_features]
#         group_ratings= pd.concat([group_ratings,selected_users_rating], axis=0)
#
#     # Aggregation if duplicates are found
#     duplicated_items = group_ratings[group_ratings['item'].duplicated(keep=False)]
#     item_set = set(duplicated_items.item)
#     for item in item_set:
#         group_ratings = group_ratings.drop(group_ratings[group_ratings["item"] == item].index)
#         duplicated_items['rating'] = duplicated_items[duplicated_items['item']==item]['rating'].mean()
#         duplicated_items = duplicated_items.iloc[[0]]
#         group_ratings = pd.concat([group_ratings,duplicated_items], axis=0)
#
#     group_ratings['user'] = i
#     groups_ratings = pd.concat([groups_ratings, group_ratings], axis=0)

## Custom made groups

In [35]:
groups = pd.read_pickle('cf_list_group_recs_nm_add.pkl')
groups_ratings = pd.DataFrame(columns=final_item_features)
for i in range(len(groups)):
    selected_users = groups[i][0][-1]

    # Create a dataframe for particular group
    group_ratings = user_item_all_features[user_item_all_features["user"]==selected_users[0]][final_item_features]
    # Append their ratings
    for user in selected_users[1:]:
        selected_users_rating = user_item_all_features[user_item_all_features["user"]==user][final_item_features]
        selected_users_rating = selected_users_rating.fillna(0)
        group_ratings= pd.concat([group_ratings,selected_users_rating], axis=0)
    # Aggregation if duplicates are found
    duplicated_items = group_ratings[group_ratings['item'].duplicated(keep=False)]
    item_set = set(duplicated_items.item)
    for item in item_set:
        group_ratings = group_ratings.drop(group_ratings[group_ratings["item"] == item].index)
        duplicated_items['rating'] = duplicated_items[duplicated_items['item']==item]['rating'].mean()
        duplicated_items = duplicated_items.iloc[[0]]
        group_ratings = pd.concat([group_ratings,duplicated_items], axis=0)


    group_ratings['user'] = i
    groups_ratings = pd.concat([groups_ratings, group_ratings], axis=0)

groups_ratings = groups_ratings.fillna(0)
group_names = groups_ratings['user'].unique()

/var/folders/hc/hnq3mts53txg3k7xbmj90z_80000gn/T/ipykernel_14743/330989318.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  groups_ratings = pd.concat([groups_ratings, group_ratings], axis=0)


In [36]:
groups_ratings.shape

(31646, 25)

## Making a group recommender system using SVR

In [37]:
def generate_Xy_for_user(username, complete_df):
    scaler = StandardScaler()
    user_rated_items = complete_df[complete_df['user']==username]
    index_x = user_rated_items.index
    X = user_rated_items.drop(columns=['user','rating', 'item'])
    normalized_matrix = scaler.fit_transform(X)
    X =  pd.DataFrame(normalized_matrix, columns=X.columns, index=index_x)
    y = user_rated_items['rating']
    return X,y

In [38]:
X,y = generate_Xy_for_user(group_names[0],groups_ratings)
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.2, random_state=42)

/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future

In [39]:
group_svr = SVR(kernel='linear')
group_svr.fit(train_X, train_y)

/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


SVR(kernel='linear')

In [40]:
predicted_y = group_svr.predict(test_X)
item_rating = test_y.to_frame().copy()
item_rating['predicted'] = predicted_y

/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [41]:
item_rating

,rating,predicted
667053,5.0,5.601860
234516,8.0,6.314862
647,10.0,6.231593
156817,7.0,5.952359
175376,7.5,6.385210
...,...,...
234522,8.0,6.032819
156847,5.0,4.203604
156954,4.0,4.889283
156825,4.0,5.230485


In [42]:
groups_ratings.to_csv("groups_ratings.csv")

In [46]:


num_users_test = 1
users_for_test = random.sample(list(all_user_names), num_users_test)

# ((item_rating['rating']-item_rating['predicted'])**2).mean(0)**0.5
# (abs(item_rating['rating']-item_rating['predicted'])).mean(0)

def evaluate_users(users, user_item_all_features, regressor):
    RMSEs = []
    MAEs = []
    for user in users:
        X,y = generate_Xy_for_user(user,user_item_all_features)
        train_X, test_X, train_y, test_y = train_test_split(X,y, test_size=0.2, random_state=42)

        regressor.fit(train_X, train_y)
        predicted_y = regressor.predict(test_X)

        item_ratings = test_y.to_frame().copy()
        item_ratings['predicted'] = predicted_y

        RMSEs.append(((item_ratings['rating']-item_ratings['predicted'])**2).mean(0)**0.5)
        MAEs.append((abs(item_ratings['rating']-item_ratings['predicted'])).mean(0))


    return RMSEs, MAEs, nDCG

user_item_all_features['ComAgeRec'].fillna(value=user_item_all_features['ComAgeRec'].mean(), inplace=True)
user_item_all_features['LanguageEase'].fillna(value=user_item_all_features['LanguageEase'].mean(), inplace=True)

regressor = SVR(kernel='linear')
RMSEs, MAEs, nDCG = evaluate_users(users_for_test, user_item_all_features[final_item_features], regressor)

avg_RMSE = np.mean(RMSEs)
avg_MAE = np.mean(MAEs)
print('RMSE:',round(avg_RMSE,2), ' | MAE:',round(avg_MAE,2), ' | nDCG:', nDCG)

regressor = DecisionTreeRegressor()
RMSEs, MAEs = evaluate_users(users_for_test, user_item_all_features[final_item_features], regressor)
avg_RMSE = np.mean(RMSEs)
avg_MAE = np.mean(MAEs)
print('RMSE:',round(avg_RMSE,2), ' | MAE:',round(avg_MAE,2))

/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/Vitalij/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future

RMSE: 1.48  | MAE: 1.22  | nDCG: 0.9670398055063476


ValueError: too many values to unpack (expected 2)